---
## Prophet

In [ ]:
import pandas as pd
import plotly_express as px
from prophet import Prophet
from useful_functions import *
file_path = "./results.json"

/Users/lilya/Downloads/M2 ISDS/Projects/time_series/venv_time_series/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --Tables--
series_train = pd.read_csv("series_train.csv")
series_test = pd.read_csv("series_test.csv")

In [3]:
# --training--
vars = ['date', 'sales']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
model_prophet.fit(train_data)

future1 = model_prophet.make_future_dataframe(periods=0)
forecast1 = model_prophet.predict(future1)
forecast1[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
series_train['Train_Forecast_prophet'] = forecast1['yhat'].values

# --testing--
test_data = series_test[['date']].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat']
series_test['Test_Forecast_prophet'] = y_test_pred.values

21:32:51 - cmdstanpy - INFO - Chain [1] start processing
21:32:51 - cmdstanpy - INFO - Chain [1] done processing


In [4]:
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet",
    params= 'without exogeneous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)

NameError: name 'file_path' is not defined

In [ ]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()

In [ ]:
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [ ]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.10937843920566573
1.4338755238509534e+20
265234.6544332331
-0.45950510525269395


Les pics semblent corrects, mais nous restons légèrement au-dessus.  
Les résultats du NRMSE paraissent satisfaisants.  

Essayons d'intégrer les variables exogènes dans le model :**`onpromotion`**, **`oil_price`**, **`holiday`**.

In [ ]:
# --training--
vars = ['date', 'sales','holiday','onpromotion','oil_price']
train_data = series_train[vars].reset_index(drop=True)  
train_data.rename(columns={'date': 'ds', 'sales': 'y'}, inplace=True)

model_prophet = Prophet()
# __exogs__
model_prophet.add_regressor('holiday')
model_prophet.add_regressor('onpromotion')
model_prophet.add_regressor('oil_price')


model_prophet.fit(train_data)

future2 = model_prophet.make_future_dataframe(periods=0)
# __exogs__
future2['holiday'] = train_data['holiday']
future2['onpromotion'] = train_data['onpromotion']
future2['oil_price'] = train_data['oil_price']

forecast2 = model_prophet.predict(future2)
forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
y_train_pred = forecast2['yhat']
series_train['Train_Forecast_prophet'] = y_train_pred.values

# --testing--
test_vars =['date','holiday','oil_price','onpromotion']
test_data = series_test[test_vars].reset_index(drop=True)  
test_data.rename(columns={'date': 'ds'}, inplace=True) 

forecast_test = model_prophet.predict(test_data)

y_test_pred = forecast_test['yhat'] 
series_test['Test_Forecast_prophet'] = y_test_pred.values

21:06:23 - cmdstanpy - INFO - Chain [1] start processing
21:06:23 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
nrmse, mape, mae, r2 = calculate_metrics(series_test['sales'], y_test_pred)
        
save_model_results(
    file_path=file_path,
    model_name="Prophet_exogs",
    params='with exogenous variables',
    nrmse=nrmse,
    mape=mape,
    mae=mae,
    r2=r2
)

In [ ]:
fig = px.line(series_train, x='date', y=['sales', 'Train_Forecast_prophet'],title="Predictions du train")
fig.show()

In [ ]:
fig = px.line(series_test, x='date', y=['sales', 'Test_Forecast_prophet'],title="Predictions du test")
fig.show()

In [ ]:
nrmse, mape, mae,r2 = calculate_metrics(series_test['sales'], y_test_pred)
print(nrmse)
print(mape)
print(mae)
print(r2)

0.13956577634483402
1.6086153714691614e+20
385361.76822215115
-1.376293314756996


>L'ajout de variables exogènes ne semble pas améliorer la situation.  
>Prophet semble bien capturer les pics, mais reste toujours légèrement au-dessus.


Sauvegarder les predictions

In [ ]:
#series_train.to_csv("series_train",index =False, header=True)
#series_test.to_csv("series_test",index =False, header=True)